In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm

In [6]:

from adn.data import DatasetMode, load_datasets

individuals_snp_dir = Path("data/output/SNPs/")
metadata_path = Path("data/DIVRICE_3k10M_ch1/DIVRICE_3k10M_metadata.tsv")
sequence_length = 64
train_eval_split = 0
labels_to_remove = "cA"
individuals_to_ignore = Path("data/bad_guys.txt")

ds, _ = load_datasets(
    individuals_snp_dir=Path(individuals_snp_dir),
    metadata_path=Path(metadata_path),
    sequence_length=sequence_length,
    train_eval_split=train_eval_split,
    data_ratio_to_use=1,
    mode=DatasetMode.SEQUENTIAL_FIXED_LEN,
    labels_to_remove=labels_to_remove,
    individual_to_ignore=individuals_to_ignore,
    overlaping_ratio=0.5,
)

2025-03-19 23:40:35.918 | INFO     | adn.data:load_metadata:43 - Using labels: {'GJ', 'XI'}. Excluding: {'cA'}
2025-03-19 23:40:35.932 | INFO     | adn.data:load_metadata:52 - Ignoring individuals: {'B176', 'IRIS_313-10998', 'IRIS_313-10771', 'B051', 'IRIS_313-11807', 'CX11,', 'CX243', 'B037', 'IRIS_313-11380', 'IRIS_313-11656', 'IRIS_313-10158', 'IRIS_313-9400', 'IRIS_313-11286', 'CX282', 'CX367', 'IRIS_313-11153', 'IRIS_313-10054', 'CX12,', 'IRIS_313-10268', 'IRIS_313-9348', 'IRIS_313-9817', 'IRIS_313-7725', 'IRIS_313-10300', 'CX49,', 'B155', 'IRIS_313-9066,', 'IRIS_313-9841', 'IRIS_313-11808', 'IRIS_313-9600', 'CX284', 'B047', 'IRIS_313-10318'}


Loading SNP data...: 100%|██████████| 2617/2617 [00:02<00:00, 910.24it/s]
2025-03-19 23:40:38.917 | INFO     | adn.data:load_datasets:103 - Train test split set to 0, using all data for training
2025-03-19 23:40:38.921 | INFO     | adn.data:load_datasets:142 - Train dataset loaded with 2617 individuals


In [ ]:
from adn.prediction import Predictor


predictor = Predictor(
    model_path=Path("output/v6-better-impl/checkpoints/checkpoint-13950")
)

predictor.predict_and_save(ds)

In [20]:
from adn.prediction_results import OnDiskPredictionResults


res = OnDiskPredictionResults(Path("output/v6-better-impl/predictions"))
res.compute_centroids()

Computing centroids: 100%|██████████| 2616/2616 [00:02<00:00, 997.45file/s] 


{'GJ': array([-0.15266877,  0.60078591, -0.66176388, -0.53418803,  0.75451188,
         0.43997152,  0.79698815, -0.5850042 , -0.56288576, -0.68005521,
         0.73142856,  0.24131174, -0.51285192,  0.49770434, -0.573842  ,
         0.79659709,  0.77155816, -0.46190751,  0.46193294,  0.78031865,
         0.6096664 , -0.16692752,  0.62156539,  0.7363393 ,  0.23007669,
         0.45648042, -0.68750784,  0.57880025, -0.54732948,  0.59418871,
         0.47536443,  0.66814083, -0.32155889, -0.79046085, -0.5609789 ,
        -0.10356427, -0.75982815, -0.3252614 , -0.72000046, -0.02559406,
        -0.65126965, -0.16488702,  0.59791346,  0.32145186, -0.33981246,
        -0.55572965, -0.56860968,  0.4696181 ,  0.53993629,  0.65417486,
        -0.69410564,  0.78017097, -0.74392477,  0.5863992 ,  0.60382635,
        -0.58398624, -0.57999358,  0.65164546, -0.29043294,  0.45882827,
         0.72835219,  0.11701948,  0.68673153,  0.4778626 ,  0.79915554,
        -0.13605611, -0.73200585, -0.38440699

In [18]:
ds.metadata_df

,Variety name,GroupK9,Country,Region,GroupK4,Region_T,Status,Comments
individual,,,,,,,,
IRIS_313-9839,COLUSA/NANO F 31,GJ-tmp,Spain,Eu,GJ,Europe,improved,NaN
IRIS_313-11005,PULU LOTONG,GJ-trp,Indonesia,As5i,GJ,Asia-South East insular-Indonesia,traditional,NaN
IRIS_313-8502,M 102,GJ-tmp,United States of America,Am,GJ,America,improved,improved outlier
IRIS_313-11156,KHUDWANI ACC 202,GJ-adm,India,As2a,GJ,Asia-India-Sub-continent,improved,NaN
IRIS_313-12346,LAB,GJ-sbtrp,Laos,As4,GJ,Asia-South East continental,traditional,NaN
...,...,...,...,...,...,...,...,...
IRIS_313-11869,MA WEI ZHAN,XI-1A,China,As6,XI,Asia-Eastern,NaN,NaN
IRIS_313-11121,GBRA,GJ-trp,Liberia,Af1,GJ,Africa-West,NaN,NaN
IRIS_313-11162,SABRAN,XI-1B,Indonesia,As5i,XI,Asia-South East insular-Indonesia,traditional,NaN


In [15]:
res.individual_to_label

{0: 'GJ',
 1: 'GJ',
 2: 'GJ',
 3: 'GJ',
 4: 'GJ',
 5: 'XI',
 6: 'XI',
 7: 'XI',
 8: 'XI',
 9: 'XI',
 10: 'XI',
 11: 'GJ',
 12: 'GJ',
 13: 'XI',
 14: 'XI',
 15: 'XI',
 16: 'XI',
 17: 'GJ',
 18: 'GJ',
 19: 'XI',
 20: 'GJ',
 21: 'XI',
 22: 'XI',
 23: 'XI',
 24: 'XI',
 25: 'XI',
 26: 'XI',
 27: 'XI',
 28: 'XI',
 29: 'XI',
 30: 'GJ',
 31: 'GJ',
 32: 'GJ',
 33: 'XI',
 34: 'XI',
 35: 'GJ',
 36: 'GJ',
 37: 'GJ',
 38: 'XI',
 39: 'GJ',
 40: 'GJ',
 41: 'XI',
 42: 'XI',
 43: 'XI',
 44: 'GJ',
 45: 'GJ',
 46: 'GJ',
 47: 'XI',
 48: 'GJ',
 49: 'XI',
 50: 'XI',
 51: 'XI',
 52: 'XI',
 53: 'XI',
 54: 'GJ',
 55: 'GJ',
 56: 'XI',
 57: 'GJ',
 58: 'GJ',
 59: 'GJ',
 60: 'XI',
 61: 'XI',
 62: 'XI',
 63: 'XI',
 64: 'XI',
 65: 'XI',
 66: 'XI',
 67: 'GJ',
 68: 'XI',
 69: 'GJ',
 70: 'XI',
 71: 'XI',
 72: 'XI',
 73: 'XI',
 74: 'XI',
 75: 'GJ',
 76: 'XI',
 77: 'XI',
 78: 'XI',
 79: 'GJ',
 80: 'XI',
 81: 'XI',
 82: 'XI',
 83: 'XI',
 84: 'XI',
 85: 'XI',
 86: 'XI',
 87: 'GJ',
 88: 'XI',
 89: 'XI',
 90: 'GJ',
 91: 'XI'